In [1]:
import sys
print(sys.executable)

/home/user/ml_env/bin/python


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7MGcGjNozVjlwdHeLsSy")
project = rf.workspace("cophe").project("sperm-motion-detection")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [3]:
!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=300 imgsz=640 plots=True

Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7925MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/user/ml_env/Sperm-Motion-Detection-1/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

In [16]:
!yolo task=detect mode=val \
    model=/home/user/ml_env/runs/detect/Train/weights/best.pt \
    data={dataset.location}/data.yaml \
    imgsz=640 \
    batch=16 \
    conf=0.25 \
    iou=0.6 \
    plots=True \
    save_txt \
    save_conf \
    name=Validate


73082.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7925MiB)
YOLO11s summary (fused): 238 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs
val: Scanning /home/user/ml_env/Sperm-Motion-Detection-1/valid/labels.cache... 5
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all       5834     130500      0.997      0.991      0.994      0.952
                     0       5781     121768      0.994       0.99      0.995      0.928
                     1       1991       4327          1          1      0.995      0.995
                     2       2665       4405      0.996      0.983      0.992      0.933
Speed: 0.2ms preprocess, 4.5ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/validation_results
💡 Learn more at https://docs.ultralytics.com/modes/val


In [17]:
!yolo task=detect mode=predict \
    model=/home/user/ml_env/runs/detect/Train/weights/best.pt \
    conf=0.25 \
    source={dataset.location}/test/images \
    save=True \
    save_txt=True \
    save_conf=True \
    project=/home/user/ml_env/runs/detect \
    name=Test \
    exist_ok=True

73440.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7925MiB)
YOLO11s summary (fused): 238 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs

image 1/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1014_jpg.rf.8cff2e466e2d17c2ed62369d73315d43.jpg: 640x640 36 0s, 2 1s, 3.4ms
image 2/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1015_jpg.rf.1a27c8324a3153d56ed97b61200941fd.jpg: 640x640 36 0s, 2 1s, 3.3ms
image 3/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1039_jpg.rf.19215e7fa6fa44f7b2dcb04657cf7146.jpg: 640x640 36 0s, 2 1s, 3.3ms
image 4/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_103_jpg.rf.d196da1046f95effe6abde2e4b4f67b8.jpg: 640x640 40 0s, 3.3ms
image 5/2939 /home/user/ml_env/Sperm-Motion-Detection-1/test/images/11_frame_1072_jpg.rf.eca5c732c627d68b5ffc0559d3306216.jpg: 640x640 36 0s, 2 1s, 3.3ms
image 6/2939 /home/user/ml_env/Sperm-Motion-Detection

In [35]:
import os
import shutil

# Source directory where model results are saved
source_dir = '/home/user/ml_env/runs/detect/'

# Destination directory for organized datasets
dest_dir = '/media/user/data/Mano/Mano-Project/Project-Mano/Output'

# List of dataset numbers
datasets = [11, 12, 13, 14, 15, 19, 21, 22, 23, 24, 29, 30, 35, 36, 38, 47, 52, 54, 60, 82]

for dataset in datasets:
    dataset_str = f"{dataset:02d}"
    dataset_dir = os.path.join(dest_dir, dataset_str)
    
    # Create dataset directories
    os.makedirs(os.path.join(dataset_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, 'labels_ftid'), exist_ok=True)

    frame_list = []

    # Process files
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.startswith(f"{dataset_str}_frame_"):
                src_path = os.path.join(root, file)
                if file.endswith('.jpg'):
                    dst_path = os.path.join(dataset_dir, 'images', file)
                    shutil.copy2(src_path, dst_path)
                    frame_list.append(dst_path)
                elif file.endswith('.txt') and not file.endswith('_with_ftid.txt'):
                    dst_path = os.path.join(dataset_dir, 'labels', file)
                    shutil.copy2(src_path, dst_path)
                elif file.endswith('_with_ftid.txt'):
                    new_name = file.replace('_with_ftid.txt', '.txt')
                    dst_path = os.path.join(dataset_dir, 'labels_ftid', new_name)
                    shutil.copy2(src_path, dst_path)

    # Create dataset.txt file
    with open(os.path.join(dataset_dir, f'{dataset_str}.txt'), 'w') as f:
        for frame_path in sorted(frame_list):
            f.write(f'{frame_path}\n')

    print(f'Organized dataset {dataset_str}')

print('All datasets organized successfully.')


Organized dataset 11
Organized dataset 12
Organized dataset 13
Organized dataset 14
Organized dataset 15
Organized dataset 19
Organized dataset 21
Organized dataset 22
Organized dataset 23
Organized dataset 24
Organized dataset 29
Organized dataset 30
Organized dataset 35
Organized dataset 36
Organized dataset 38
Organized dataset 47
Organized dataset 52
Organized dataset 54
Organized dataset 60
Organized dataset 82
All datasets organized successfully.
